In [23]:
import os
import pandas as pd
import json

from pathlib import Path

In [28]:
delays = pd.read_csv("data/delays_on_stations.csv", index_col=0)
meteo = pd.read_csv("data/synop_fin.csv", index_col=0)

delays["departure_time"] = pd.to_datetime(delays["departure_time"])
meteo["datetime"] = pd.to_datetime(meteo["datetime"])

display(delays.tail())
display(meteo.head())

/Users/jakubcwynar/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,departure_time,departure_delay,latitude,longitude
13249,2018-06-06 16:45:00,17.0,53.067135,18.692424
13250,2018-06-06 18:45:00,20.0,49.878771,18.688364
13251,2018-06-06 18:45:00,19.0,49.886916,18.758581
13252,2018-06-06 18:45:00,19.0,49.892758,18.809291
13253,2018-06-06 21:15:00,19.0,54.038333,22.948804


,datetime,station_name,latitude,longitude,temperature,rainfall,wind_speed
0,2017-12-10 00:00:00,KOŁOBRZEG-DŹWIRZYNO,54.156846,15.402596,2.40,0.10,4.00
1,2017-12-10 00:15:00,KOŁOBRZEG-DŹWIRZYNO,54.156846,15.402596,2.14,0.06,3.67
2,2017-12-10 00:30:00,KOŁOBRZEG-DŹWIRZYNO,54.156846,15.402596,1.96,0.04,3.39
3,2017-12-10 00:45:00,KOŁOBRZEG-DŹWIRZYNO,54.156846,15.402596,1.84,0.01,3.17
4,2017-12-10 01:00:00,KOŁOBRZEG-DŹWIRZYNO,54.156846,15.402596,1.80,-0.00,3.00


In [24]:
DATE_FROM = "2018-06-01"
DATE_TO = "2018-06-06"

MAP_DIR = Path(os.path.abspath(""))
RAIN_IMG = MAP_DIR / "images" / "rain.png"

In [98]:
DEFAULT_MARKER_SIZE = 0.1

def delay_to_geojson(row):
    _, data = row
    timestamp = int(data["departure_time"].timestamp()) * 1e+3
    lat, lon = data["latitude"], data["longitude"]
    radius = DEFAULT_MARKER_SIZE * data["departure_delay"]
    
    point = {
      "type": "Feature",
      "properties": {
        "times": [
          timestamp
        ],
        "iconstyle": {
          "fillColor": "red",
          "stroke": True,
          "radius": radius 
        },
        "icon": "circle"
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          lon,
          lat
        ]
      }
    }
    return point
    

def rain_to_geojson(row):
    _, data = row
    timestamp = int(data["datetime"].timestamp()) * 1e+3
    lat, lon = data["latitude"], data["longitude"]
    size = 12
    rainfall = data["rainfall"]
    icon_size = [size * rainfall] * 2
    
    point = {
      "type": "Feature",
      "properties": {
        "times": [
          timestamp
        ],
        "iconstyle": {
          "iconUrl": str(RAIN_IMG),
          "iconSize": icon_size
        },
        "icon": "marker"
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          lon,
          lat
        ]
      }
    }
    return point

In [99]:
delays_range = (delays["departure_time"] >= DATE_FROM) & (delays["departure_time"] < DATE_TO)
valid_latlon = delays["latitude"] != float("inf")

meteo_range = (meteo["datetime"] >= DATE_FROM) & (meteo["datetime"] < DATE_TO)
was_rainy = (meteo["rainfall"] > 0)

delays = delays[delays_range & valid_latlon]
meteo = meteo[meteo_range & was_rainy]

In [100]:
delays.shape

(12255, 4)

In [101]:
delay_gjson = {
  "type": "FeatureCollection",
  "features": [
    delay_to_geojson(point) for point in delays.iterrows()
  ]
}

meteo_gjson = {
  "type": "FeatureCollection",
  "features": [
    rain_to_geojson(point) for point in meteo.iterrows()
  ]
}

In [102]:
with open("delays.json", "w") as f:
    json.dump(delay_gjson, f, indent=True)
    
with open("meteo.json", "w") as f:
    json.dump(meteo_gjson, f)